# Part 2 Instructions

## Have Customers Narrow Their Travel Seaches Based on Temperature and Precipitation

In [1]:
# Import dependencies
import gmaps
from config import g_key
import pandas as pd
import requests

In [2]:
# Import WeatherPy_challenge.csv as a new DataFrame
challenge_gmaps_df = pd.read_csv('data/WeatherPy_challenge.csv')
challenge_gmaps_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Weather Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Kulhudhuffushi,6.62,73.07,84.15,79,100,18.95,MV,2020-05-11 04:14:59,moderate rain,0.0,0.0
1,1,Campbellsville,37.34,-85.34,46.40,61,1,4.70,US,2020-05-11 04:15:00,clear sky,0.0,0.0
2,2,Qaanaaq,77.48,-69.36,6.89,76,0,6.04,GL,2020-05-11 04:13:48,clear sky,0.0,0.0
3,3,Porto Novo,6.50,2.60,77.00,88,40,2.24,BJ,2020-05-11 04:15:01,scattered clouds,0.0,0.0
4,4,Port Alfred,-33.59,26.89,62.01,75,0,1.01,ZA,2020-05-11 04:15:02,clear sky,0.0,0.0


In [3]:
# Prompt the user for inputs: min/max temp range, rain/snow
min_prompt_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_prompt_temp = float(input('What is the maximum temperature you would like for your trip? '))                    
raining_prompt = input("Do you want it to be raining? (yes/no) ")
snowing_prompt = input("Do you want it to be snowing? (yes/no) ")

# Filter DataFrames based on input and nested decision statements, and logical expressions
if (raining_prompt == 'yes') and (snowing_prompt == 'no') :
    preferred_challenges_df = challenge_gmaps_df.loc[(challenge_gmaps_df['Rain inches (last 3 hrs)']!= 0) &\
                                                     (challenge_gmaps_df['Snow inches (last 3 hrs)']== 0) &\
                                                             (challenge_gmaps_df['Max Temp']<= max_prompt_temp) & \
                                                            (challenge_gmaps_df['Max Temp']>= min_prompt_temp)]
elif (raining_prompt == 'no') and (snowing_prompt == 'no') :
    preferred_challenges_df = challenge_gmaps_df.loc[(challenge_gmaps_df['Rain inches (last 3 hrs)']== 0) &\
                                                     (challenge_gmaps_df['Snow inches (last 3 hrs)']== 0) &\
                                                             (challenge_gmaps_df['Max Temp']<= max_prompt_temp) & \
                                                            (challenge_gmaps_df['Max Temp']>= min_prompt_temp)]
elif (raining_prompt == 'no') and (snowing_prompt == 'yes') :
    preferred_challenges_df = challenge_gmaps_df.loc[(challenge_gmaps_df['Rain inches (last 3 hrs)']== 0) &\
                                                     (challenge_gmaps_df['Snow inches (last 3 hrs)']!= 0) &\
                                                             (challenge_gmaps_df['Max Temp']<= max_prompt_temp) & \
                                                            (challenge_gmaps_df['Max Temp']>= min_prompt_temp)]
else:
    preferred_challenges_df = challenge_gmaps_df.loc[(challenge_gmaps_df['Rain inches (last 3 hrs)']!= 0) &\
                                                     (challenge_gmaps_df['Snow inches (last 3 hrs)']!= 0) &\
                                                             (challenge_gmaps_df['Max Temp']<= max_prompt_temp) & \
                                                            (challenge_gmaps_df['Max Temp']>= min_prompt_temp)]
    
preferred_challenges_df.head()

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Weather Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Kulhudhuffushi,6.62,73.07,84.15,79,100,18.95,MV,2020-05-11 04:14:59,moderate rain,0.0,0.0
3,3,Porto Novo,6.50,2.60,77.00,88,40,2.24,BJ,2020-05-11 04:15:01,scattered clouds,0.0,0.0
6,6,Georgetown,5.41,100.34,87.80,79,20,4.70,MY,2020-05-11 04:01:11,few clouds,0.0,0.0
11,11,Saint-Philippe,-21.36,55.77,71.60,60,75,6.93,RE,2020-05-11 04:14:44,broken clouds,0.0,0.0
13,13,Sao Filipe,14.90,-24.50,72.91,74,4,7.07,CV,2020-05-11 04:13:23,clear sky,0.0,0.0


In [4]:
# Clean up the DataFrame by dropna() funtion
clean_preferred_challenges_df = preferred_challenges_df.dropna()
clean_preferred_challenges_df.count()
# make a copy for a new DataFrame 
challenge_hotel_df = clean_preferred_challenges_df[["City", "Country","Max Temp",  "Weather Description","Lat", "Lng"]].copy()
challenge_hotel_df['Hotel Name'] = ''

# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}

# search nearest hotel name by using google place API 
for index, row in challenge_hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    # Add the latitude and longitude to location key for the params dictionary.
    params['location'] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    try: 
        r_hotels = requests.get(base_url, params= params).json()

        challenge_hotel_df.loc[index, 'Hotel Name'] = r_hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping')

# Save and upload the new DataFrame as csv file
challenge_hotel_df.to_csv(path_or_buf='data/WeatherPy_vacation.csv', index_label='City_ID')

challenge_hotel_df.head(10)

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... s

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Kulhudhuffushi,MV,84.15,moderate rain,6.62,73.07,Haajy Guest House 1
3,Porto Novo,BJ,77.00,scattered clouds,6.50,2.60,Tour Eiffel Hotel Benin
6,Georgetown,MY,87.80,few clouds,5.41,100.34,Cititel Penang
11,Saint-Philippe,RE,71.60,broken clouds,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
13,Sao Filipe,CV,72.91,clear sky,14.90,-24.50,Tortuga B&B
15,Arys,KZ,73.40,few clouds,42.43,68.80,"Gostinitsa ""Madani"""
17,Vila Do Maio,CV,71.60,scattered clouds,15.13,-23.22,Residence 'I Delfini'
19,Goundam,ML,87.35,scattered clouds,16.41,-3.67,
21,Touros,BR,75.20,scattered clouds,-5.20,-35.46,INN NEW HORIZON
22,Mahebourg,MU,73.40,broken clouds,-20.41,57.70,Shandrani Beachcomber Resort & Spa


In [5]:
# Create a marker layer map with a pop-up info box by gmaps dependence
gmaps.configure(api_key = g_key)
locations = challenge_hotel_df[['Lat','Lng']]

# Adding pop-up markers
info_box_template='''
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp}°F</dd>
</dl>'''

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in challenge_hotel_df.iterrows() ]

fig = gmaps.figure(center = (30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))